In [1]:
!pip install hopsworks

Processing /Users/jayden/Library/Caches/pip/wheels/76/b1/d1/e8ec69a796901e1dadc9409b4d91da6b7b37ccb3b7a97a05ef/hopsworks-3.4.3-py3-none-any.whl
  Using cached boto3-1.33.9-py3-none-any.whl (139 kB)
  Using cached hsfs-3.4.5.tar.gz (170 kB)
  Using cached pyjks-20.0.0-py2.py3-none-any.whl (45 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached furl-2.1.3-py2.py3-none-any.whl (20 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached mock-5.1.0-py3-none-any.whl (30 kB)
  Using cached pyhumps-1.6.1-py3-none-any.whl (5.0 kB)
Processing /Users/jayden/Library/Caches/pip/wheels/97/2d/ce/5495c785662eb34365396458884fa6a08aa7d91de9889b046d/hsml-3.4.5-py3-none-any.whl
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.8.2-py3-none-any.whl (82 kB)
  Using cached botocore-1.33.9-py3-none-any.whl (11.8 MB)
  Using cached pandas-2.0.3-cp39-cp39-macosx_10_9_x86_64.whl (11.8 MB)
  Using cached numpy-1.26.2-cp39-cp39-macosx_10_9_x86_

In [2]:
import hopsworks
project = hopsworks.login()

/Users/jayden/Documents/Resources/Y4S1-KTH/ID2223/Labs/Project/inference-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197787


In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib
import gradio as gr

def determine_valuation(row):
    tolerance = 0.05  # 5% tolerance
    if row['Actual'] < row['Predicted'] * (1 - tolerance):
        return 'UNDERVALUED'
    elif row['Actual'] > row['Predicted'] * (1 + tolerance):
        return 'OVERVALUED'
    else:
        return 'FAIR-VALUED'

def valuate(file):
    df = pd.read_csv(file)
    q1 = df['list_price'].quantile(0.25)
    q3 = df['list_price'].quantile(0.75)
    iqr = q3 - q1
    filtered_df = df.query('(@q1 - 1.5 * @iqr) <= list_price <= (@q3 + 1.5 * @iqr)')
    q1 = filtered_df['sqft'].quantile(0.25)
    q3 = filtered_df['sqft'].quantile(0.75)
    iqr = q3 - q1
    filtered_df = filtered_df.query('(@q1 - 1.5 * @iqr) <= sqft <= (@q3 + 1.5 * @iqr)')
    q1 = filtered_df['price_per_sqft'].quantile(0.25)
    q3 = filtered_df['price_per_sqft'].quantile(0.75)
    iqr = q3 - q1
    filtered_df = filtered_df.query('(@q1 - 1.5 * @iqr) <= price_per_sqft <= (@q3 + 1.5 * @iqr)')
    q1 = filtered_df['stories'].quantile(0.25)
    q3 = filtered_df['stories'].quantile(0.75)
    iqr = q3 - q1
    filtered_df = filtered_df.query('(@q1 - 1.5 * @iqr) <= stories <= (@q3 + 1.5 * @iqr)')
    filtered_df = filtered_df.dropna(subset=['sqft', 'list_price', 'price_per_sqft', 'stories', 'year_built'])
    filtered_df = filtered_df.dropna(subset=['sqft', 'list_price', 'price_per_sqft', 'stories', 'year_built'])
    feature_df = filtered_df.drop(columns=['property_url', 'mls', 'mls_id', 'status', 'street', 'unit', 'last_sold_date', 'sold_price', 'days_on_mls', 'primary_photo', 'alt_photos'])
    feature_df.fillna(0, inplace=True)
    le = LabelEncoder()
    le.fit(feature_df['style'])
    encoded_labels = le.transform(feature_df['style'])
    feature_df['style'] = encoded_labels
    le.fit(feature_df['city'])
    encoded_labels = le.transform(feature_df['city'])
    feature_df['city'] = encoded_labels
    le.fit(feature_df['state'])
    encoded_labels = le.transform(feature_df['state'])
    feature_df['state'] = encoded_labels
    le.fit(feature_df['list_date'])
    encoded_labels = le.transform(feature_df['list_date'])
    feature_df['list_date'] = encoded_labels    
    true_values = feature_df['list_price']
    feature_df = feature_df.drop('list_price', axis=1)

    mr = project.get_model_registry()
    model = mr.get_model("xgboost_model", version=3)
    model_dir = model.download()
    model = joblib.load(model_dir + "/xgboost_model.pkl")

    predicted_val = model.predict(feature_df)

    result = pd.DataFrame({'Property URL': filtered_df['property_url'], 'Predicted': predicted_val, 'Actual': true_values})
    result['valuation'] = result.apply(determine_valuation, axis=1)

    return result

iface1 = gr.Interface(
    fn=valuate,
    inputs=gr.File(type='filepath'),
    outputs="dataframe",
    title="New York Residential Property Valuation",
    description="Identify Undervalued Properties in New York!",
)

gr.TabbedInterface(
    [iface1], ["New York"]
).launch()

        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


Connected. Call `.close()` to terminate connection gracefully.


[18:06:06] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.


Connected. Call `.close()` to terminate connection gracefully.


[18:08:15] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.


Connected. Call `.close()` to terminate connection gracefully.


[18:08:41] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
